## Change just here the sequence length that you want 

Ctrl+f "path_to_change" to adjust the paths and make the code work 

In [37]:
sequence_length_left = 2000
sequence_length_right = 2000


# Les deux listes
liste_nc = [
    'NC_000001.11', 'NC_000002.12', 'NC_000003.12', 'NC_000004.12', 'NC_000005.10',
    'NC_000006.12', 'NC_000007.14', 'NC_000008.11', 'NC_000009.12',
    'NC_000010.11', 'NC_000011.10', 'NC_000012.12', 'NC_000013.11',
    'NC_000014.9', 'NC_000015.10', 'NC_000016.10', 'NC_000017.11',
    'NC_000018.10', 'NC_000019.10', 'NC_000020.11', 'NC_000021.9',
    'NC_000022.11', 'NC_000023.11', 'NC_000024.10'
]

liste_chr = [
    'chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8',
    'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16',
    'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX',
    'chrY'
]

# Création du dictionnaire dans l'ordre demandé
dex_to_ncbi = dict(zip(liste_chr, liste_nc))

import pandas as pd
from Bio import SeqIO

# Charger le fichier FASTA et stocker les séquences dans un dictionnaire
fasta_file = "../ncbi_dataset/data/GCF_000001405.40/GCF_000001405.40_GRCh38.p14_genomic.fna" #path_to_change genome_code
chromosome_sequences = {}
for record in SeqIO.parse(fasta_file, "fasta"):
    chromosome_sequences[record.id] = str(record.seq)

# Charger le DataFrame
df = pd.read_csv("dex_annotations_important.csv") #path_to_change the annotations file
df_filtered = df[df['seqname'] != 'chrM']


# Fonction pour extraire la séquence centrée autour de 'start' ou 'end' basé sur 'strand'
def get_centered_sequence(row):
    chromosome = dex_to_ncbi[row['seqname']]
    if row['strand'] == '+':
        center = row['start']
    else:
        center = row['end']
    
    seq = chromosome_sequences.get(chromosome, "")
    if seq:
        start_idx = max(0, center - sequence_length_left)
        end_idx = min(len(seq), center + sequence_length_right)
        return seq[start_idx-1:end_idx]
    return ""

# Ajouter la nouvelle colonne avec les séquences extraites
df_filtered['centred_sequence'] = df_filtered.apply(get_centered_sequence, axis=1)

def reverse_complement(sequence):
    # Dictionnaire de complémentarité des nucléotides
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C',
                  'a': 't', 't': 'a', 'c': 'g', 'g': 'c',
                  'N': 'N', 'n': 'n'}  # 'N' reste 'N', 'n' reste 'n'
    
    # Générer la séquence complémentaire
    complement_sequence = ''.join(complement.get(base, base) for base in sequence)
    
    # Inverser la séquence complémentaire
    reverse_complement_sequence = complement_sequence[::-1]
    
    return reverse_complement_sequence

df_filtered['centred_sequence_'] = df_filtered.apply(lambda row: reverse_complement(row['centred_sequence'].upper()) if row['strand'] == '-' else row['centred_sequence'], axis=1)
df_filtered.drop(columns = ["centred_sequence"], inplace = True)
df_filtered.rename(columns = {"centred_sequence_" : "centred_sequence"}, inplace = True)

pituitary = pd.read_csv("final_result.csv") # path_to_change gene_expression data from gtex 

final_df = pd.merge(df_filtered, pituitary, left_on="GENE_ID", right_on='gene_id')

# Replace zeros with a small value to avoid errors in log transformation
final_df['Pituitary'] = final_df['Pituitary'].replace(0, 0.001)

# Take the logarithm base 10 of the 'Pituitary' column
final_df['Pituitary'] = np.log10(final_df['Pituitary'])

# Select desired columns
final_df = final_df[['centred_sequence', 'Pituitary', 'last_gene_id', 'GENE_ID', 'dex_gene_id']]

# Convert 'centred_sequence' to uppercase
final_df['centred_sequence'] = final_df['centred_sequence'].str.upper()

final_df


/var/tmp/ipykernel_13740/4118683251.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['centred_sequence'] = df_filtered.apply(get_centered_sequence, axis=1)
/var/tmp/ipykernel_13740/4118683251.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['centred_sequence_'] = df_filtered.apply(lambda row: reverse_complement(row['centred_sequence'].upper()) if row['strand'] == '-' else row['centred_sequence'], axis=1)
/var/tmp/ipykernel_13740/4118683251.py:72: SettingWithCopyWarning: 
A va

,centred_sequence,Pituitary,last_gene_id,GENE_ID,dex_gene_id
0,TGACTTGCTGTCTCTAGATAGTTGCTGCAAGTCTAAAAAAACTTGA...,0.849062,ENSG00000118473.23,ENSG00000118473,ENSG00000118473.17
1,TCTTTGAGCTCCAGACCCAAGTGTCTGGTGGCCTAGAGGCTGCGAA...,1.273982,ENSG00000162426.16,ENSG00000162426,ENSG00000162426.10
2,CAGCAGTTTCACTCCTATGTATACACTCAAGAAAATGGAAAACAGG...,1.500136,ENSG00000157191.20,ENSG00000157191,ENSG00000157191.15
3,ATGGTTCAAGCCCCCATCTAAGGGGAGCCTCCTGGAATTTATTTAT...,1.298134,ENSG00000169504.15,ENSG00000169504,ENSG00000169504.10
4,ACTCCTATGCATAGAAAACAAGATGGTTCCAGGCAGAAGGGCCCAG...,0.825821,ENSG00000142920.18,ENSG00000142920,ENSG00000142920.12
...,...,...,...,...,...
21861,GAAAAATACTGTAATTGGCATAAAATATAAATTGTGATGGGACACC...,1.946064,ENSG00000008735.14,ENSG00000008735,ENSG00000008735.10
21862,GTTCAGGGACCCACAGCAAAGTCTGTCTAACTAGACGCATCCGATC...,1.810176,ENSG00000100299.18,ENSG00000100299,ENSG00000100299.13
21863,CTGCCTTCTGGGTTCAAATGATTCTCCTGCCTCAGCTTTCTGAGTA...,0.013292,ENSG00000100312.11,ENSG00000100312,ENSG00000100312.6
21864,ACATAAAAATGTTTTATAGATTGGACTAAAGACCTTTGAGGTCTCT...,1.421082,ENSG00000079974.19,ENSG00000079974,ENSG00000079974.13


In [38]:
import os
import shutil

def create_or_empty_directory(dir_path):
    if os.path.exists(dir_path):
        # If directory exists, delete all its contents
        shutil.rmtree(dir_path)
    # Create the directory (or recreate it if it was deleted)
    os.makedirs(dir_path)


if sequence_length_left > 0 and sequence_length_right > 0:
    path_ = f"DATASETS/GRCH38/{sequence_length_left}_tss_{sequence_length_right}"
elif sequence_length_left > 0:
    path_ = f"DATASETS/GRCH38/{sequence_length_left}_tss"
elif sequence_length_right > 0:
    path_ = f"DATASETS/GRCH38/tss_{sequence_length_right}"
else:
    path_ = "DATASETS/GRCH38/tss"

create_or_empty_directory(path_)

final_df.to_csv(f"{path_}/{path_.split('/')[-1]}.csv", index=False)

## Format the dataset for DNABERT-2 method 

This code will generate the csv files that can be feeded to dnabert-2 method

In [39]:

import pandas as pd
from sklearn.model_selection import train_test_split

# Split dataset into training and temp sets
train_df, temp_df = train_test_split(final_df, test_size=0.3, random_state=42)

# Split temp set into testing and development sets
test_df, dev_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Save the datasets to CSV files (optional)
create_or_empty_directory(path_+"/dnabert_2")


train_df.to_csv(f"{path_}/dnabert_2/train.csv", index=False)
test_df.to_csv(f"{path_}/dnabert_2/test.csv", index=False)
dev_df.to_csv(f"{path_}/dnabert_2/dev.csv", index=False)

print("Datasets have been split and saved.")


Datasets have been split and saved.


## Format the data for DExTER method

This code make sure that the split of data is the same with the one being used with grch37 (the data sent by dexter authors) you can eliminate this caracteristics easily 

In [40]:
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os

# Chargement du fichier CSV contenant les IDs et les séquences
csv_file = 'DATASETS/GRCH38/2000_tss_2000/2000_tss_2000.csv'
csv_data = pd.read_csv(csv_file)

# Création d'un dictionnaire pour accéder facilement aux séquences par leur ID (avant le point)
sequence_dict = {}
for index, row in csv_data.iterrows():
    seq_id = row['last_gene_id'].split('.')[0]
    sequence_dict[seq_id] = row['centred_sequence']

# Lecture du fichier FASTA envoyé par les auteurs de Dexter et remplacement des séquences 
fasta_file = 'send_by_dexter_guys/ncbiRefSeqCurated.hg19.2kb.around.geneStart.ENSEMBLgeneID.fa'
fasta_records = list(SeqIO.parse(fasta_file, 'fasta'))

# Liste pour stocker les nouveaux enregistrements FASTA
new_fasta_records = []

# Remplacement des séquences dans les enregistrements FASTA
for record in fasta_records:
    gene_id = record.id.split('.')[0]
    if gene_id in sequence_dict:
        new_seq = Seq(sequence_dict[gene_id])
        new_record = SeqRecord(new_seq, id=record.id, description=record.description)
        new_fasta_records.append(new_record)

# Définir le chemin du fichier de sortie
output_fasta_file = 'DATASETS/GRCH38/2000_tss_2000/dexter/dexter_grch38_genes.fa'

# Créer le répertoire si nécessaire
os.makedirs(os.path.dirname(output_fasta_file), exist_ok=True)

# Écriture des enregistrements modifiés dans un nouveau fichier FASTA avec des séquences sur une seule ligne
with open(output_fasta_file, 'w') as output_handle:
    for record in new_fasta_records:
        output_handle.write(f">{record.id}\n")
        output_handle.write(str(record.seq) + "\n")

print(f"Les séquences ont été remplacées et enregistrées dans {output_fasta_file}")


Les séquences ont été remplacées et enregistrées dans DATASETS/GRCH38/2000_tss_2000/dexter/dexter_grch38_genes.fa


now we have our sequences orgnised as the data sent by dexter authors we need the expression data for dexter method we have only to give the expression data over all the tissues and there are algorithms that gonna associate each sequence with its corresponding expression with the tissue

In [41]:
import shutil

# Chemins des fichiers source et destination

source_file = 'send_by_dexter_guys/GTEx_gene_median_7tissues.tsv'
destination_file = 'DATASETS/GRCH38/2000_tss_2000/dexter/GTEx_gene_median_7tissues.tsv'
os.makedirs(os.path.dirname(destination_file), exist_ok=True)

# Copier le fichier
shutil.copy(source_file, destination_file)

'DATASETS/GRCH38/2000_tss_2000/dexter/GTEx_gene_median_7tissues.tsv'